# Hands On

## Prepare

```SQL
DROP TABLE IF EXISTS lp_flags_map CASCADE;
CREATE TABLE lp_flags_map(id INT, description VARCHAR(40));
INSERT INTO lp_flags_map VALUES 
(0, 'Unused'),
(1, 'Normal'),
(2, 'Redirect'),
(3, 'Dead');

DROP TABLE IF EXISTS tbl CASCADE;
SELECT generate_series AS id, 0 AS data
INTO tbl
FROM generate_series(1,300);

ALTER TABLE tbl SET (autovacuum_enabled = false);

DROP VIEW IF EXISTS vw_inspect_tbl_first_page;
CREATE VIEW vw_inspect_tbl_first_page AS
SELECT
	lp AS line_pointer
	,lp_off AS liner_pointer_offset
	,lp_len AS liner_pointer_length
	,lp_off - lp_len AS next_liner_pointer_offset
	,t_xmin
	,t_xmax
	,t_ctid
	,f.description AS liner_pointer_flags
FROM 
	heap_page_items(get_raw_page('tbl', 0)) hp
	LEFT JOIN lp_flags_map f ON hp.lp_flags = f.id;

DROP VIEW IF EXISTS vw_inspect_tbl_second_page;
CREATE VIEW vw_inspect_tbl_second_page AS
SELECT
	lp AS line_pointer
	,lp_off AS liner_pointer_offset
	,lp_len AS liner_pointer_length
	,lp_off - lp_len AS next_liner_pointer_offset
	,t_xmin
	,t_xmax
	,t_ctid
	,f.description AS liner_pointer_flags
FROM 
	heap_page_items(get_raw_page('tbl', 1)) hp
	LEFT JOIN lp_flags_map f ON hp.lp_flags = f.id;
```

```SQL
VACUUM tbl;
```

### VM And FSM

```SQL
-- Check the visibility
SELECT * FROM pg_visibility('tbl'::regclass);
-- Nothing... because it's not created yet
```

```BASH
# Bash!
# Let's find the physical layout of our new table
RELATION_FILE_PATH=$(psql -t -c "SELECT pg_relation_filepath('tbl');" | tr -d ' ')
echo "Relation file path is: '$RELATION_FILE_PATH'"
RELATION_LOCATION=$(echo $RELATION_FILE_PATH | cut -d "/" -f 1,2)
RELATION_FILE_NODE=$(echo $RELATION_FILE_PATH | cut -d "/" -f 3)
echo "Relation locations is: '$RELATION_LOCATION' and file node is: '$RELATION_FILE_NODE'"

# Check the fork
ls $PGDATA/$RELATION_LOCATION | grep $RELATION_FILE_NODE # Looks like there is only the main data file for now

# Perform a VACUUM
psql -c "VACUUM tbl;"

ls $PGDATA/$RELATION_LOCATION | grep $RELATION_FILE_NODE # Now we have vm and fsm added
```

```SQL
-- After first VACUUM
SELECT * FROM pg_visibility('tbl'::regclass);
```

## Dead Tuples

```SQL
DELETE FROM tbl WHERE id BETWEEN 1 AND 5;
SELECT * FROM vw_inspect_tbl_first_page; -- Still `normal` flag
```

```SQL
INSERT INTO tbl VALUES (1001,0), (1002,0);
SELECT * FROM vw_inspect_tbl_first_page; -- Still!
SELECT * FROM vw_inspect_tbl_second_page; -- No reuse
```

```SQL
UPDATE tbl SET data = 1 WHERE id = 6; 
SELECT * FROM vw_inspect_tbl_first_page; -- This retrieves a row and triggers the in-page maintenance, some reuse

INSERT INTO tbl VALUES (1003,0), (1004,0);
SELECT * FROM vw_inspect_tbl_first_page; -- Insert still can't use it
SELECT * FROM vw_inspect_tbl_second_page; -- No reuse
```

```SQL
VACUUM tbl
```

```SQL
SELECT * FROM vw_inspect_tbl_first_page; -- Now the tuples are unused and not dead
INSERT INTO tbl VALUES (1005,0), (1006,0);
SELECT * FROM vw_inspect_tbl_first_page; -- Insert can reuse place
```

## Relation Size

### Insert More Tuples

```SQL
INSERT INTO tbl
SELECT generate_series AS id, 0 AS data
FROM generate_series(1,1000000);
```

```SQL
SELECT pg_size_pretty(pg_relation_size('tbl'));
```

### Delete Tuples

```SQL
DELETE FROM tbl WHERE id BETWEEN 300 AND 900000;
SELECT pg_size_pretty(pg_relation_size('tbl')); -- Still big
```

```SQL
VACUUM tbl;
SELECT pg_size_pretty(pg_relation_size('tbl')); -- What? still big...
```

```SQL
VACUUM FULL tbl;
SELECT pg_size_pretty(pg_relation_size('tbl')); -- Finally...
```